In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
import train_model as tm
import visualize as vis
import build_features as bf
import make_dataset as md

In [ ]:
# set random state
rand_state = 1

# Load files into memory

In [ ]:
counts_filename = 'data/count_data_species_raw_WIS_overlapping_fungi_bacteria_12773samples.tsv'
metadata_filename = 'data/metadata_species_WIS_overlapping_fungi_bacteria_12773samples.tsv'

In [ ]:
counts = md.read_fungi_data(counts_filename)
metadata = md.read_fungi_data(metadata_filename)

In [ ]:
metadata = metadata.replace('Not available', np.nan)

In [ ]:
# consider selecting metadata columns before combining
combined = pd.merge(metadata, counts, on="sampleid", how="inner")

In [ ]:
metadata.shape

In [ ]:
counts.shape

In [ ]:
combined.shape

In [ ]:
combined["days_to_death"].value_counts()

In [ ]:
combined["days_to_death"].mean()

In [ ]:
# note 76% of samples dont have DTD
combined["days_to_death"].isna().mean()

In [ ]:
# 3036 data points to work with
combined["days_to_death"].notna().sum()

In [ ]:
# drop nans + values < 10k
plt.boxplot(combined[combined['days_to_death'] < 10_000]['days_to_death'].dropna(), vert=False)

## Searching for metadata features

In [ ]:
plt.hist(metadata['analyte_amount'], bins=100)
plt.show()

In [ ]:
# experimental_strategy          - ohe
# analyte_amount                 - numerical (normalize?)
# analyte_A260A280Ratio          - numerical (contains zero?)
# aliquot_concentration          - numerical (omit 2.10 value)
# gender                         - ohe (55 nan)
# race                           - ohe (1462 nan, keep)
# ethnicity                      - ohe (3096 nan)
# disease_type                   - ohe (32 types)
# sample_type                    - ordinal [['Primary Tumor', 'Recurrent Tumor', 'Additional - New Primary', 'Metastatic', 'Blood Derived Normal', 'Solid Tissue Normal']]
# primary_site                   - ohe
# age_at_diagnosis               - numerical
# reference_genome               - ohe
# data_submitting_center_label   - ohe
# investigation                  - ohe
# days_to_death                  - numerical
# tissue_source_site_label       - ordinal (avoid ohe, 179 unique vals)
# country_of_sample_procurement  - ohe
# pathologic_t_label             - ohe (reduce stages?)
# pathologic_n_label             - ohe (reduce stages?)
# histological_diagnosis_label   - ordinal (avoid ohe, 71 unique vals)
# pathologic_stage_label         - ohe (reduce stages)
# PlateCenter                    - numerical
# PlateCenterFlag                - ohe

important_cols = ['experimental_strategy', 'analyte_amount', 'analyte_A260A280Ratio', 'aliquot_concentration', 'gender', 'race', 'ethnicity', 
'disease_type', 'sample_type','primary_site', 'age_at_diagnosis', 'reference_genome', 'data_submitting_center_label', 
'investigation', 'tissue_source_site_label', 'country_of_sample_procurement', 'pathologic_t_label', 'pathologic_n_label', 
'histological_diagnosis_label', 'pathologic_stage_label', 'PlateCenter', 'PlateCenterFlag', 'days_to_death']

## Metadata Regression

In [ ]:
ohe_feats = ['experimental_strategy', 'gender', 'race', 'ethnicity', 'disease_type', 'primary_site',
             'reference_genome', 'data_submitting_center_label', 'investigation', 'country_of_sample_procurement', 
             'pathologic_t_label', 'pathologic_n_label', 'pathologic_stage_label', 'PlateCenterFlag']
scaler_feats = ['analyte_amount', 'analyte_A260A280Ratio', 'aliquot_concentration']
passthrough_feats = ['age_at_diagnosis', 'PlateCenter']
ordinal_feats = ['sample_type', 'tissue_source_site_label', 'histological_diagnosis_label']
drop_feats = ['sample_name', 'run_prefix', 'cgc_base_name',
              'filename', 'cgc_id', 'cgc_filename', 'vital_status',
              'data_subtype', 'tcga_sample_id', 'cgc_case_uuid', 'cgc_platform',
              'gdc_file_uuid', 'cgc_sample_uuid',
              'cgc_aliquot_uuid', 'tcga_aliquot_id',
              'tcga_case_id', 'days_to_death', 'knightlabID', 'portion_is_ffpe']

In [ ]:
ct = make_column_transformer(    
    (StandardScaler(), scaler_feats), 
    (OneHotEncoder(), ohe_feats),
    (OrdinalEncoder, ordinal_feats),
    ("passthrough", passthrough_feats),
    ("drop", drop_feats),
)

In [ ]:
ct.fit(metadata)

## Regression Model

In [ ]:
reg_data = combined[combined["days_to_death"].notna()]

In [ ]:
reg_data.shape

In [ ]:
# separate X and Y and generate 

test_prop = 0.1
reg_X = reg_data.drop(columns=metadata.columns)
reg_Y = reg_data["days_to_death"]
reg_Xtrain, reg_Xtest, reg_Ytrain, reg_Ytest = train_test_split(reg_X, reg_Y, test_size=test_prop, random_state=rand_state)

In [ ]:
reg_model = LinearRegression()
reg_model.fit(reg_Xtrain, reg_Ytrain)
preds = reg_model.predict(reg_Xtest)
preds = [x if x > 0 else 0 for x in preds] # Replace negative predictions
scores = mean_squared_error(reg_Ytest, preds)
scores

In [ ]:
(reg_Ytest - preds).head()